# <span style="color:green"> Below are the important steps in Exporting the Saved Model using Estimator and build_parsing_serving_input_receiver_fn:</span> 

## classifier = tf.estimator.DNNClassifier(feature_columns=my_feature_columns,    
##                                        hidden_units=[10, 10],  # Two hidden layers of 10 nodes each.
##                                            n_classes=3)

## feature_spec = tf.feature_column.make_parse_example_spec(my_feature_columns)
## serving_input_receiver_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(feature_spec)

## Export_Dir = 'Premade_Estimator_Export' #No need of Version Number

## Final_Export_Dir = classifier.export_savedmodel(Export_Dir, serving_input_receiver_fn)

## Final_Export_Dir = Export_Dir + Folder Created by Current TimeStamp

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import argparse
import iris_data

In [2]:
BATCH_SIZE = 100
STEPS = 1000
Export_Dir = 'Premade_Estimator_Export' #No need of Version Number

In [3]:
(train_x, train_y), (test_x, test_y) = iris_data.load_data()

# Feature columns describe how to use the input.
my_feature_columns = []
for key in train_x.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))

In [4]:
# Build a DNN with 2 hidden layers and 10 nodes in each hidden layer.
classifier = tf.estimator.DNNClassifier(feature_columns=my_feature_columns,    
                                        hidden_units=[10, 10],  # Two hidden layers of 10 nodes each.
                                            n_classes=3) # The model must choose between 3 classes.    

W0814 19:06:38.758629 140127005255488 estimator.py:1811] Using temporary folder as model directory: /tmp/tmp7m7t1q5q


In [5]:
# Train the Model.
classifier.train(input_fn=lambda:iris_data.train_input_fn(train_x, train_y, BATCH_SIZE),steps=STEPS)

W0814 19:06:38.776670 140127005255488 deprecation.py:323] From /usr/local/google/home/mothukuru/anaconda3/envs/TF_PY_36/lib/python3.6/site-packages/tensorflow/python/training/training_util.py:236: Variable.initialized_value (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
W0814 19:06:38.804061 140127005255488 deprecation.py:506] From /usr/local/google/home/mothukuru/anaconda3/envs/TF_PY_36/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0814 19:06:39.450907 140127005255488 deprecation.py:323] From /usr/local/google/home/mo

In [6]:
eval_result = classifier.evaluate(input_fn=lambda:iris_data.eval_input_fn(test_x, test_y, BATCH_SIZE))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

W0814 19:06:41.260846 140127005255488 deprecation.py:323] From /usr/local/google/home/mothukuru/anaconda3/envs/TF_PY_36/lib/python3.6/site-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.



Test set accuracy: 0.967



In [7]:
# Generate predictions from the model
expected = ['Setosa', 'Versicolor', 'Virginica']
predict_x = {
    'SepalLength': [5.1, 5.9, 6.9],
    'SepalWidth': [3.3, 3.0, 3.1],
    'PetalLength': [1.7, 4.2, 5.4],
    'PetalWidth': [0.5, 1.5, 2.1],
}

predictions = classifier.predict(input_fn=lambda:iris_data.eval_input_fn(features = predict_x, labels = None, 
                                            batch_size=BATCH_SIZE))

In [8]:
template = ('\nPrediction is "{}" ({:.1f}%), expected "{}"')

for pred_dict, expec in zip(predictions, expected):
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print(template.format(iris_data.SPECIES[class_id],100 * probability, expec))


Prediction is "Setosa" (99.9%), expected "Setosa"

Prediction is "Versicolor" (99.7%), expected "Versicolor"

Prediction is "Virginica" (97.5%), expected "Virginica"


### Saving the Model for Serving

In [9]:
feature_spec = tf.feature_column.make_parse_example_spec(my_feature_columns)
print(feature_spec)

W0814 19:06:41.769943 140127005255488 deprecation.py:323] From /usr/local/google/home/mothukuru/anaconda3/envs/TF_PY_36/lib/python3.6/site-packages/tensorflow/python/feature_column/feature_column.py:806: NumericColumn._parse_example_spec (from tensorflow.python.feature_column.feature_column_v2) is deprecated and will be removed in a future version.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.


{'SepalLength': FixedLenFeature(shape=(1,), dtype=tf.float32, default_value=None), 'SepalWidth': FixedLenFeature(shape=(1,), dtype=tf.float32, default_value=None), 'PetalLength': FixedLenFeature(shape=(1,), dtype=tf.float32, default_value=None), 'PetalWidth': FixedLenFeature(shape=(1,), dtype=tf.float32, default_value=None)}


In [10]:
serving_input_receiver_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(feature_spec)
export_dir = classifier.export_savedmodel(Export_Dir, serving_input_receiver_fn)
print('Exported to {}'.format(export_dir))

W0814 19:06:41.775899 140127005255488 deprecation.py:323] From <ipython-input-10-924d12de5068>:2: Estimator.export_savedmodel (from tensorflow_estimator.python.estimator.estimator) is deprecated and will be removed in a future version.
Instructions for updating:
This function has been renamed, use `export_saved_model` instead.
W0814 19:06:41.905965 140127005255488 deprecation.py:323] From /usr/local/google/home/mothukuru/anaconda3/envs/TF_PY_36/lib/python3.6/site-packages/tensorflow/python/saved_model/signature_def_utils_impl.py:145: build_tensor_info (from tensorflow.python.saved_model.utils_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.


Exported to b'Premade_Estimator_Export/1565789801'


# Model is stored in /usr/local/google/home/mothukuru/Jupyter_Notebooks/export

In [11]:
#print(serving_input_receiver_fn.func_globals)

In [12]:
!ls -R Premade_Estimator_Export/1562576539

Premade_Estimator_Export/1562576539:
saved_model.pb	variables

Premade_Estimator_Export/1562576539/variables:
variables.data-00000-of-00002  variables.data-00001-of-00002  variables.index


In [13]:
!saved_model_cli show --dir Premade_Estimator_Export/1562576539 --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['classification']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['inputs'] tensor_info:
        dtype: DT_STRING
        shape: (-1)
        name: input_example_tensor:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['classes'] tensor_info:
        dtype: DT_STRING
        shape: (-1, 3)
        name: dnn/head/Tile:0
    outputs['scores'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 3)
        name: dnn/head/predictions/probabilities:0
  Method name is: tensorflow/serving/classify

signature_def['predict']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['examples'] tensor_info:
        dtype: DT_STRING
        shape: (-1)
        name: input_example_tensor:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['all_class_ids'] tensor_info:
        dtype: DT_

In [14]:
!saved_model_cli show --dir Premade_Estimator_Export/1562083639 \
  --tag_set serve --signature_def serving_default

Traceback (most recent call last):
  File "/usr/local/google/home/mothukuru/anaconda3/envs/TF_PY_36/bin/saved_model_cli", line 10, in <module>
    sys.exit(main())
  File "/usr/local/google/home/mothukuru/anaconda3/envs/TF_PY_36/lib/python3.6/site-packages/tensorflow/python/tools/saved_model_cli.py", line 909, in main
    args.func(args)
  File "/usr/local/google/home/mothukuru/anaconda3/envs/TF_PY_36/lib/python3.6/site-packages/tensorflow/python/tools/saved_model_cli.py", line 621, in show
    _show_inputs_outputs(args.dir, args.tag_set, args.signature_def)
  File "/usr/local/google/home/mothukuru/anaconda3/envs/TF_PY_36/lib/python3.6/site-packages/tensorflow/python/tools/saved_model_cli.py", line 133, in _show_inputs_outputs
    tag_set)
  File "/usr/local/google/home/mothukuru/anaconda3/envs/TF_PY_36/lib/python3.6/site-packages/tensorflow/python/tools/saved_model_utils.py", line 113, in get_meta_graph_def
    saved_model = read_saved_model(saved_model_dir)
  File "/usr/local/google/

In [15]:
!saved_model_cli show --dir Premade_Estimator_Export/1562083639 \
  --tag_set serve --signature_def classification

Traceback (most recent call last):
  File "/usr/local/google/home/mothukuru/anaconda3/envs/TF_PY_36/bin/saved_model_cli", line 10, in <module>
    sys.exit(main())
  File "/usr/local/google/home/mothukuru/anaconda3/envs/TF_PY_36/lib/python3.6/site-packages/tensorflow/python/tools/saved_model_cli.py", line 909, in main
    args.func(args)
  File "/usr/local/google/home/mothukuru/anaconda3/envs/TF_PY_36/lib/python3.6/site-packages/tensorflow/python/tools/saved_model_cli.py", line 621, in show
    _show_inputs_outputs(args.dir, args.tag_set, args.signature_def)
  File "/usr/local/google/home/mothukuru/anaconda3/envs/TF_PY_36/lib/python3.6/site-packages/tensorflow/python/tools/saved_model_cli.py", line 133, in _show_inputs_outputs
    tag_set)
  File "/usr/local/google/home/mothukuru/anaconda3/envs/TF_PY_36/lib/python3.6/site-packages/tensorflow/python/tools/saved_model_utils.py", line 113, in get_meta_graph_def
    saved_model = read_saved_model(saved_model_dir)
  File "/usr/local/google/

In [16]:
!saved_model_cli show --dir Premade_Estimator_Export/1562083639 \
  --tag_set serve --signature_def predict

Traceback (most recent call last):
  File "/usr/local/google/home/mothukuru/anaconda3/envs/TF_PY_36/bin/saved_model_cli", line 10, in <module>
    sys.exit(main())
  File "/usr/local/google/home/mothukuru/anaconda3/envs/TF_PY_36/lib/python3.6/site-packages/tensorflow/python/tools/saved_model_cli.py", line 909, in main
    args.func(args)
  File "/usr/local/google/home/mothukuru/anaconda3/envs/TF_PY_36/lib/python3.6/site-packages/tensorflow/python/tools/saved_model_cli.py", line 621, in show
    _show_inputs_outputs(args.dir, args.tag_set, args.signature_def)
  File "/usr/local/google/home/mothukuru/anaconda3/envs/TF_PY_36/lib/python3.6/site-packages/tensorflow/python/tools/saved_model_cli.py", line 133, in _show_inputs_outputs
    tag_set)
  File "/usr/local/google/home/mothukuru/anaconda3/envs/TF_PY_36/lib/python3.6/site-packages/tensorflow/python/tools/saved_model_utils.py", line 113, in get_meta_graph_def
    saved_model = read_saved_model(saved_model_dir)
  File "/usr/local/google/

### Execute the below commands in Terminal

sudo docker pull tensorflow/serving

sudo docker run -p 8501:8501 --mount type=bind,source=/usr/local/google/home/mothukuru/Jupyter_Notebooks/TF_Serving/Serving_Made_Easy/Serving_Demystified/Premade_Estimator_Export,target=/models/Premade_Estimator_Export -e MODEL_NAME=Premade_Estimator_Export -t tensorflow/serving &

### Working Inference Command

curl -d '{"examples":[{"SepalLength":[5.1],"SepalWidth":[3.3],"PetalLength":[1.7],"PetalWidth":[0.5]}]}' -X POST http://localhost:8501/v1/models/Premade_Estimator_Export:classify

**Output:**

{"results": [[["0", 0.998636663], ["1", 0.00136337487], ["2", 1.95144964e-12]]]}

### Non-working Inference Commands

curl -d '{"inputs": [5.1,3.3,1.7,0.5]}' \
  -X POST http://localhost:8501/v1/models/Premade_Estimator_Export:predict

### Explain the error with DT_STRING Type of Predict Input

## Go through the below link which comprises KEDB document for different kinds of errors which occured during Inference using REST

## https://docs.google.com/spreadsheets/d/1_uZxA2Zp6QD8_46CzXAZXCXVLuiYxybDUGnu72dO58o/edit#gid=1827007920